<a href="https://colab.research.google.com/github/HemantajayW/Stock_market_prediction_during_WARPERIOD/blob/main/Russia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 11.0 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Loading main data

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
data = yf.download("RNO.PA" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.head(3)
data.index.strftime('%Y-%m-%d')

Index(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05', '2018-01-08',
       '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12', '2018-01-15',
       ...
       '2022-04-28', '2022-04-29', '2022-05-02', '2022-05-03', '2022-05-04',
       '2022-05-05', '2022-05-06', '2022-05-09', '2022-05-10', '2022-05-11'],
      dtype='object', name='Date', length=1117)

In [ ]:
# Sort the data points based on indexes just for confirmation 
data.sort_index(inplace = True)

In [ ]:
# Remove any duplicate index 
data = data.loc[~data.index.duplicated(keep='first')]

In [ ]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-05-09,23.040001,23.270000,22.455000,22.455,22.455,1281289
2022-05-10,22.940001,22.980000,22.315001,22.375,22.375,1257446
2022-05-11,22.625000,23.120001,22.400000,22.945,22.945,1254869


In [ ]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,84.000000,84.129997,81.529999,83.800003,70.844910,872516
2018-01-03,84.040001,84.720001,83.919998,84.089996,71.090065,773646
2018-01-04,84.739998,85.599998,84.610001,85.220001,72.045380,1149993
2018-01-05,85.500000,87.320000,85.440002,87.199997,73.719276,926891
2018-01-08,87.510002,88.900002,87.330002,88.470001,74.792946,1022507


In [ ]:
# Check for missing values 
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1117.000000,1117.000000,1117.000000,1117.000000,1117.000000,1.117000e+03
mean,46.166041,46.814064,45.456609,46.096855,42.497140,1.621359e+06
std,21.558392,21.674978,21.435734,21.542264,17.204872,1.030720e+06
min,15.250000,16.000000,12.770000,14.550000,13.733418,0.000000e+00
25%,30.150000,30.620001,29.700001,30.100000,30.020000,9.665210e+05
50%,37.680000,38.150002,36.840000,37.634998,37.174999,1.350736e+06
75%,59.529999,60.330002,58.779999,59.680000,53.205009,1.913758e+06
max,99.279999,100.800003,97.760002,98.750000,83.483704,9.603937e+06


Understanding Trends with in the Data

In [ ]:
import plotly.graph_objects as go

In [ ]:
# Check the trend in Closing Values 
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [ ]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)
data.index

DatetimeIndex(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
               '2018-01-08', '2018-01-09', '2018-01-10', '2018-01-11',
               '2018-01-12', '2018-01-15',
               ...
               '2022-04-28', '2022-04-29', '2022-05-02', '2022-05-03',
               '2022-05-04', '2022-05-05', '2022-05-06', '2022-05-09',
               '2022-05-10', '2022-05-11'],
              dtype='datetime64[ns]', name='Date', length=1117, freq=None)

Scrapping extra information

In [ ]:
import requests 

response = requests.get('https://www.alphavantage.co/query?function=RSI&symbol=RNO.PA&interval=daily&time_period=5&series_type=close&apikey=43T9T17VCV2ME4SM') 
response = response.json()

In [ ]:
response.keys()

dict_keys(['Meta Data', 'Technical Analysis: RSI'])

In [ ]:
rsi_data = pd.DataFrame.from_dict(response['Technical Analysis: RSI'] , orient='index')

In [ ]:
rsi_data.head()

,RSI
2005-01-10,77.0284
2005-01-11,78.4825
2005-01-12,67.7597
2005-01-13,54.8738
2005-01-14,64.1663


In [ ]:
rsi_data = rsi_data[rsi_data.index >= '2018-01-01']
rsi_data.index

Index(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05', '2018-01-08',
       '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12', '2018-01-15',
       ...
       '2022-04-28', '2022-04-29', '2022-05-02', '2022-05-03', '2022-05-04',
       '2022-05-05', '2022-05-06', '2022-05-09', '2022-05-10', '2022-05-11'],
      dtype='object', length=1117)

In [ ]:
rsi_data['RSI'] = rsi_data['RSI'].astype(np.float64)

In [ ]:
rsi_data.head()

rsi_data.index

Index(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05', '2018-01-08',
       '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12', '2018-01-15',
       ...
       '2022-04-28', '2022-04-29', '2022-05-02', '2022-05-03', '2022-05-04',
       '2022-05-05', '2022-05-06', '2022-05-09', '2022-05-10', '2022-05-11'],
      dtype='object', length=1117)

In [ ]:
print(data.shape)
# data=pd.merge(data,rsi_data,how='inner',on=["Date"])
bata = data.join(rsi_data)
bata.shape

(1117, 2)


(1117, 3)

In [ ]:
# print(data.shape)
data.head()

,Close,Volume
Date,,
2018-01-02,83.800003,872516
2018-01-03,84.089996,773646
2018-01-04,85.220001,1149993
2018-01-05,87.199997,926891
2018-01-08,88.470001,1022507


In [ ]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2021-09-01')].shape[0]
test_length

179

In [ ]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [ ]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/1085 [00:00<?, ?it/s]

In [ ]:
# Check the shapes
X.shape , Y.shape

((1085, 32, 2), (1085,))

In [ ]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [ ]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((906, 32, 2), (906,))

In [ ]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((179, 32, 2), (179,))

In [ ]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:,:,i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [ ]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [ ]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [ ]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [ ]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=5, min_lr=0.00001,verbose = 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional , BatchNormalization

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 3))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [ ]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 10,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
91/91 [==============================] - 154s 2s/step - loss: 0.0523 - val_loss: 0.0030 - lr: 0.0020
Epoch 2/10
91/91 [==============================] - 132s 1s/step - loss: 0.0257 - val_loss: 0.0025 - lr: 0.0020
Epoch 3/10
91/91 [==============================] - 136s 1s/step - loss: 0.0203 - val_loss: 0.0019 - lr: 0.0020
Epoch 4/10
91/91 [==============================] - 135s 1s/step - loss: 0.0168 - val_loss: 0.0015 - lr: 0.0020
Epoch 5/10
91/91 [==============================] - 133s 1s/step - loss: 0.0137 - val_loss: 0.0013 - lr: 0.0020
Epoch 6/10
91/91 [==============================] - 135s 1s/step - loss: 0.0114 - val_loss: 0.0011 - lr: 0.0020
Epoch 7/10
91/91 [==============================] - 135s 1s/step - loss: 0.0093 - val_loss: 0.0010 - lr: 0.0020
Epoch 8/10
91/91 [==============================] - 132s 1s/step - loss: 0.0078 - val_loss: 8.7480e-04 - lr: 0.0020
Epoch 9/10
91/91 [==============================] - 132s 1s/step - loss: 0.0074 - val_loss: 7.9602e-

In [ ]:
# Load the best weights
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [ ]:
Predictions = model.predict(Xtest)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [ ]:
Predictions.shape

(179, 1)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
# Check the Predictions vs Actual
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [ ]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [ ]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [ ]:
Predictions = model.predict(Total_features)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)
Predictions.shape

(1085,)

In [ ]:
metric = tf.keras.metrics.Accuracy()
metric.update_state(Actual,Predictions)


<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1085.0>

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [ ]:
# Save and Load the whole model
model.save("Model.h5")
loaded_model = tf.keras.models.load_model("Model.h5")

Realtime Prediction

In [ ]:
def PredictStockPrice(Model , DataFrame , PreviousDate , feature_length = 32):
    idx_location = DataFrame.index.get_loc(PreviousDate)
    Features = DataFrame.iloc[idx_location - feature_length : idx_location,:].values
    Features = np.expand_dims(Features , axis = 0)
    Features = Feature_Scaler.transform(Features)
    Prediction = Model.predict(Features)
    Prediction = Target_Scaler.inverse_transform(Prediction)
    return Prediction[0][0]

In [ ]:
PredictStockPrice(loaded_model , data , '2021-01-14')

35.410378